# Workshop n°9: Draw variable roof

# Roberto Nunziato, matricola 438778

# Descrizione

L'obiettivo di questo workshop è quello di riuscire a costruire un tetto, a prescindere dal poligono di base che ne costituisce il perimetro. Nella risoluzione si è seguito il seguente approccio:     

1) Si determina il valore degli angoli interni del poligono    
2) Si verifica quali sono gli angoli convessi del poligono         
3) Si costruisce la base del tetto       
3) Per ciascuna coppia di segmenti adiacenti del poligono, si costruiscono le facce delle falde        
4) Si determinano i punti di incrocio delle facce, che rappresentano i vertici delle falde poste ad un'altezza fissata      
5) Si costuisce il tetto finale    

# Librerie importate

In [9]:
from pyplasm import *
from fpformat import *
import math

# Codice per la realizzazione del modello finale

# Funzione 1: creaListaAngoli

La seguente funzione prende in input la lista dei vertici che compongono il poligono di base e resituisce, in una lista, il valore in gradi degli angoli interni del poligono

In [10]:
"""
    @param listaVertici: array di vertici che compongono il poligono di base
    @return listaAngoli: lista degli angoli
"""
def creaListaAngoli(listaVertici):
    
    #Nella listaArcTan, in corrispondenza dell'indice del segmento memorizzo il valore
    #dell'arcotangente 
    listaArcTan = []
    listaAngoli = []
    for index in range(0,len(listaVertici)-1):
        x = listaVertici[index+1][0]-listaVertici[index][0]
        y = listaVertici[index+1][1]-listaVertici[index][1]
        #Uso la funzione atan2 che calcola l'arcotangente, dati x ed y (math.degrees converte la misura in gradi)
        arctan = math.degrees(math.atan2(y,x))
        if arctan + 180 <= 180:
            listaArcTan.append([arctan,arctan+180])
        else:
            listaArcTan.append([arctan,arctan-180])
    
    #Mi scorro la lista delle arcotangenti e mi calcolo il valore effettivo dell'angolo 
    for index in range(0,len(listaArcTan)-1):
        angolo = listaArcTan[index][1]-listaArcTan[index+1][0]
        #Se l'angolo è negativo, gli sommo 360°
        if angolo < 0:
            listaAngoli.append(angolo+360)
        else:
            listaAngoli.append(angolo)
    #Elaboro l'ultimo elemento della lista
    angolo = listaArcTan[len(listaArcTan)-1][1]-listaArcTan[0][0]
    if angolo < 0:
        listaAngoli.append(angolo+360)
    else:
        listaAngoli.append(angolo)
    return listaAngoli


# Funzione 2: listaAngoliConvessi

La seguente funzione prende in input la lista dei valori degli angoli interni del poligono e restituisce la lista degli angoli convessi

In [11]:
"""
    @param listaAngoli: angoli calcolati dalla funzione precedente
    @return angoliConvessi: lista degli indici in corrispondenza dei quali ci sono angoli convessi
"""
#Funzione che, data la lista degli angoli, restituisce in una lista gli angoli del poligono che sono convessi
def listaAngoliConvessi(listaAngoli):
    angoliConvessi = []
    for index in range(0,len(listaAngoli)-1):
        #math.fabs restituisce il valore assoluto
        if math.fabs(listaAngoli[index]) > 180:
            angoliConvessi.append(index+1)
    return angoliConvessi

# Funzione 3: disegnaBaseTetto

La seguente funzione prende in input la lista dei vertici che compongono il poligono di base, la lista degli angoli convessi e crea la base del tetto

In [12]:
"""
    @param vertici: lista dei vertici che compongono il poligono di base
    @param angoliConvessi: lista degli indici degli angoli convessi del poligono
    @return lista,vertici: lista contiene i vertici non usati nella costruzione
"""   
#Funzione che, data la lista dei vertici del poligono e la lista degli angoli convessi, crea la base del tetto
def disegnaBaseTetto(vertici,angoliConvessi):
    lista = []
    del vertici[-1]
    if len(angoliConvessi) > 0:
        for index in angoliConvessi:
            if index == 1:
                lista.append([1,2,len(vertici)])
            else:
                lista.append([index-1,index,index+1])
    #Lista dei vertici inclusi nei piani aggiuntivi
    verticiUsati = []
    for elem in lista[0]:
        verticiUsati.append(elem)
    #Lista dei vertici non inclusi
    verticiNonUsati = []
    for elem in range(1,len(vertici)):
        if elem not in verticiUsati:
            verticiNonUsati.append(elem)
            
    verticiNonUsati.append(verticiUsati[len(verticiUsati)-2])
    verticiNonUsati.append(verticiUsati[len(verticiUsati)-1])
    lista.append(verticiNonUsati)
    return [lista, vertici]

# Funzione 4: creaPiano

La seguente funzione prende in input la lista dei vertici che compongono il poligono di base, l'altezza a cui verrà posta la falda e restituisce la lista di tutti i piani (vengono costruiti i piani dei segmenti adiacenti)

In [13]:
"""
    @param vertici: lista dei vertici che compongono il poligono di base
    @param h: altezza a cui verrà posta la falda
    @return piani: lista dei piani costruiti per segmenti adiacenti del poligono
"""   
def creaPiano(vertici,h):
    piani = []
    #Valuto l'orientamento dei segmenti nel piano
    for index in range(0,len(vertici)-1):
        v1 = vertici[index]
        v2 = vertici[index+1]
        #se l'ascissa del secondo punto è maggiore, il segmento è rivolto verso l'alto
        if v1[0]<v2[0]:
            m = (v2[1]-v1[1])/(v2[0]-v1[0])
            #Valuto la pendenza, prendendo un valore costante
            if math.fabs(m)<5:
                v3 = [v1[0]-1,v1[1]+1]  
                v4 = [v2[0],v2[1]+1]
            else:
                if m>0:    
                    v3 = [v1[0]-1,v1[1]+1]
                    v4 = [v2[0]-1,v2[1]+1]
                else:
                    v3 = [v1[0]+1,v1[1]+1]
                    v4 = [v2[0]+1,v2[1]+1]
        else:
            m = -1*(v2[1]-v1[1])/(v2[0]-v1[0])
            if math.fabs(m)<5:
                v3 = [v1[0]+1,v1[1]-1]   
                v4 = [v2[0],v2[1]-1]
            else:
                if m>0:    
                    v3 = [v1[0]-1,v1[1]-1]
                    v4 = [v2[0]-1,v2[1]-1]
                else:
                    v3 = [v1[0]+1,v1[1]-1]
                    v4 = [v2[0]+1,v2[1]-1]
        punti = [v1,v2,v3,v4]
        celle = [[1,2,3,4]]
        piano = MKPOL([punti,celle,None])
        piani.append(piano)
    piani.append(piani[0])    
    return piani

# Funzione 5: intersezionePiani

La seguente funzione prende in input la lista dei piani del poligono costruita dalla funzione precedente, calcola l'intersezione tra i piani e ne restituisce i vertici in una lista

In [14]:
"""
    @param listaPiani: lista dei piani
    @return puntiInterni: lista punti di intersezione tra piani adiacenti del poligono
"""   
#Funzione che, data la lista dei piani, restituisce i punti di intersezione che comporranno i punti interni
#del tetto
def intersezionePiani(listaPiani):
    puntiInterni = []
    for index in range(0,len(listaPiani)-1):
        intersezione = INTERSECTION([listaPiani[index],listaPiani[index+1]])
        punti = UKPOL(SKEL_0(intersezione))[0]
        puntiPiano1 = UKPOL(SKEL_2(listaPiani[index]))[0]
        puntiPiano2 = UKPOL(SKEL_2(listaPiani[index+1]))[0]
        puntiIntersezione = []
        puntiPiano1puliti = []
        puntiPiano2puliti = []
        
        for p in puntiPiano1:
            l = []
            for x in p:
                l.append(float(fix(x,3)))
            puntiPiano1puliti.append(l)

        for p in puntiPiano2:
            l = []
            for x in p:
                l.append(float(fix(x,3)))
            puntiPiano2puliti.append(l)

        for p in punti:
            l = []
            for x in p:
                l.append(float(fix(x,3)))
            puntiIntersezione.append(l)         
        puntiIntersezioneInterni = []
        for p in puntiIntersezione:
            if p not in puntiPiano2 and p not in puntiPiano1:
                puntiIntersezioneInterni.append(p)
        #Calcolo il punto medio dei punti di interesse da usare come vertice interno
        sommaX = 0 
        sommaY = 0
        cont = 0       
        for p in puntiIntersezioneInterni:
            sommaX = sommaX + p[0]
            sommaY = sommaY + p[1]
            cont = cont + 1
        puntoMedio = [float(fix(sommaX/cont,3)),float(fix(sommaY/cont,3)),1]
        puntiInterni.append(puntoMedio)
        
    #Sposto l'ultimo elemento della lista in prima posizione
    x = puntiInterni[-1]
    puntiInterni.insert(0, x)
    return puntiInterni


# Funzione 6: disegnaTetto

La seguente funzione prende in input la lista dei vertici del poligono, l'altezza a cui deve essere posta la falda e restituisce l'oggetto HPC del tetto

In [15]:
"""
    @param listaVertici: lista dei vertici che compongono il poligono di base
    @param h: altezza a cui deve essere posta la falda
    @return oggetto HPC del tetto
"""      
def disegnaTetto(listaVertici, h):
    perimetro = STRUCT(AA(POLYLINE)([listaVertici]))
    angoli = creaListaAngoli(listaVertici)
    angoliConvessi = listaAngoliConvessi(angoli)
    baseTetto = disegnaBaseTetto(listaVertici,angoliConvessi)    
    lista = baseTetto[0]
    vertici = baseTetto[1]
    
    sottotetto = []
    sottotetto.append(MKPOL([vertici,lista,None]))

    listaVertici.insert(len(listaVertici),polygonVertex[0])

    piani = creaPiano(listaVertici,h)
    puntiInterni = intersezionePiani(piani)
    
    sovratetto = []
    sovratetto.append(MKPOL([puntiInterni,lista,None]))  
    
    falde = []#costruisco le varie falde del tetto
    falde.append(TEXTURE(['tegole.jpg',TRUE,FALSE,1,1,0,15,20]))
    
    for i in range(0, len(listaVertici)-1):
        if len(listaVertici[i])<3:
            listaVertici[i].insert(len(listaVertici[i]),0)
        if len(listaVertici[i+1])<3:
            listaVertici[i+1].insert(len(listaVertici[i+1]),0)
        punti = [listaVertici[i],listaVertici[i+1],puntiInterni[i+1],puntiInterni[i]]
        celle = [[1,2,3,4]] 
        falde.append(MKPOL([punti,celle,None]))
    return STRUCT([STRUCT(falde),STRUCT(sottotetto),STRUCT(sovratetto)])

In [16]:
#Definisco i vertici del poligono
polygonVertex = [[1.,1.],[6.,2.],[12.,1.],[15.,5.],[10.,8.],[8.,7.],[5.,5.],[1.,1.]]
tetto = disegnaTetto(polygonVertex, 1)
VIEW(tetto)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10e750b40> >

# Screenshot dell'oggetto HPC ottenuto

![text-alt](Screenshot1.png)
![text-alt](Screenshot2.png)
![text-alt](Screenshot3.png)